# LLM Domain Adaptation with ORPO, AWS Trainium and AWS Inferentia2

**SageMaker Studio**: Jupyter Lab  
**Kernel**: Python3  

This exercise is divide into 2 parts:
 - **Data prep + model alignment**
 - Model deployment + tests

In this notebook you'll run the first part. You start by preparing the dataset and uploading it to S3. Then you kick-off a SageMaker training Job that will execute our training script to do model alignment using ORPO with a Llama3.2 1B params.

In [ ]:
%pip install -U sagemaker datasets s3fs

In [ ]:
import os
import sagemaker

print(sagemaker.__version__)
if not sagemaker.__version__ >= "2.146.0": print("You need to upgrade or restart the kernel if you already upgraded")

os.makedirs("src", exist_ok=True)

## ATTENTION: Copy your HF Access token to the following variable
HF_TOKEN=""
assert HF_TOKEN != "", " >>> Go to your HF account and get an access token. Set HF_TOKEN to your token if you want to define your own cache repo"

role = sagemaker.get_execution_role()
sess = sagemaker.Session()
bucket = sess.default_bucket()
region = sess.boto_region_name

print(f"sagemaker role arn: {role}")
print(f"sagemaker bucket: {bucket}")
print(f"sagemaker session region: {region}")

## 1) Prepare the dataset
In this step we'll download a dataset from HF, get only a slice of it and then upload to S3.


In [ ]:
import datasets

num_samples=1024
idx = 147

train_dataset = datasets.load_dataset("mlabonne/orpo-dpo-mix-40k", split="all")
train_dataset = train_dataset.shuffle(seed=42).select(range(num_samples))

train_path = f"s3://{bucket}/datasets/orpo-dpo-mix-40k/train"
train_dataset.save_to_disk(train_path)
print(f"Train path: {train_path}")

df = train_dataset.to_pandas()
print("\n=== SAMPLE ===\n")
chosen,rejected,prompt,question = [df[i][idx] for i in ['chosen', 'rejected', 'prompt', 'question']]
print(f"Question: {question}\n")
print("=== CHOSEN ===\n")
for i in chosen: print(f" >> Role: {i['role']}\nContent: {i['content']}\n")

print("=== REJECTED ===\n")
for i in rejected: print(f" >> Role: {i['role']}\nContent: {i['content']}\n") 

train_dataset.to_pandas().head(10)

## 2) Create training artifacts
### 2.1) Dependencies descriptor
Installing the libraries listed in this file will be the first thing SageMaker will do.

In [ ]:
%%writefile src/requirements.txt
--extra-index-url https://pip.repos.neuron.amazonaws.com
git+https://github.com/huggingface/optimum-neuron@02c331d
trl==0.11.4
peft==0.13.2

### 2.1) Training script
Please note the arguments passed to this script are the **hyperparameters** defined in the SageMaker Estimator (next section)

In [ ]:
%%writefile src/train.py
# Copyright Amazon.com, Inc. or its affiliates. All Rights Reserved.
# SPDX-License-Identifier: MIT-0

import os
import torch
import argparse
from peft import (
    LoraConfig,
    TaskType,
    get_peft_model
)
from trl import setup_chat_format
from huggingface_hub import login
from datasets import load_from_disk
from transformers import AutoModelForCausalLM, AutoTokenizer
from optimum.neuron import NeuronORPOConfig, NeuronORPOTrainer

if __name__=='__main__':
    parser = argparse.ArgumentParser()

    # hyperparameters sent by the client are passed as command-line arguments to the script.
    parser.add_argument("--epochs", type=int, default=1)
    parser.add_argument("--num_samples", type=int, default=32)
    parser.add_argument("--max_seq_len", type=int, default=256)
    parser.add_argument("--max_prompt_len", type=int, default=128)
    parser.add_argument("--train_batch_size", type=int, default=1)
    parser.add_argument("--eval_batch_size", type=int, default=1)
    parser.add_argument("--tp_size", type=int, default=1)
    parser.add_argument("--pp_size", type=int, default=1)
    
    parser.add_argument("--model_id", type=str, required=True)
    parser.add_argument("--zero_1", type=bool, default=True)
    parser.add_argument("--learning_rate", type=float, default=5e-5)
    parser.add_argument("--weight_decay", type=float, default=0.01)
    parser.add_argument("--bf16", type=bool, default=True)

    # Data, model, and output directories
    parser.add_argument("--output_data_dir", type=str, default=os.environ.get("SM_OUTPUT_DATA_DIR", "output"))
    parser.add_argument("--model_dir", type=str, default=os.environ.get("SM_MODEL_DIR", "model"))

    parser.add_argument("--training_dir", type=str, default=os.environ.get("SM_CHANNEL_TRAIN", None))

    parser.add_argument("--hf_token", type=str, default=None)

    args, _ = parser.parse_known_args()

    if not args.hf_token is None and len(args.hf_token) > 0:
        print("HF token defined. Logging in...")
        login(token=args.hf_token)

    model = AutoModelForCausalLM.from_pretrained(args.model_id)
    peft_config = LoraConfig(
        r=16,
        lora_alpha=32,
        lora_dropout=0.05,
        bias="none",
        task_type="CAUSAL_LM",
        target_modules=['up_proj', 'down_proj', 'gate_proj', 'k_proj', 'q_proj', 'v_proj', 'o_proj']
    )
    tokenizer = AutoTokenizer.from_pretrained(args.model_id)
    model, tokenizer = setup_chat_format(model, tokenizer)
    
    print(f"Loading dataset...")
    train_dataset = load_from_disk(args.training_dir)

    def format_chat_template(row):
        row["chosen"] = tokenizer.apply_chat_template(row["chosen"], tokenize=False)
        row["rejected"] = tokenizer.apply_chat_template(row["rejected"], tokenize=False)
        return row
        
    train_dataset = train_dataset.map(
        format_chat_template,
        num_proc=os.cpu_count(),
    )
    
    training_args = NeuronORPOConfig(
        max_length=args.max_seq_len,
        max_prompt_length=args.max_prompt_len,
        beta=0.1,
        
        zero_1=args.zero_1,
        bf16=args.bf16,
        tensor_parallel_size=args.tp_size,
        pipeline_parallel_size=args.pp_size,
        
        #eval_strategy="epoch",
        learning_rate=args.learning_rate,
        weight_decay=args.weight_decay,

        num_train_epochs=args.epochs,
        output_dir=args.output_data_dir,
        overwrite_output_dir=True,

        per_device_train_batch_size=args.train_batch_size,
        #per_device_eval_batch_size=args.eval_batch_size,

        gradient_accumulation_steps=1,
        #eval_accumulation_steps=1,

        logging_dir=f"{args.output_data_dir}/logs",
        logging_strategy="steps",
        logging_steps=10,
        save_steps=50,
        max_grad_norm=1,
        save_strategy="steps",
        save_total_limit=1,
        remove_unused_columns=False,
        hub_token=args.hf_token
    )

    trainer = NeuronORPOTrainer(
        model=model,
        args=training_args,
        tokenizer=tokenizer,
        peft_config=peft_config,
        train_dataset=train_dataset
    )
    trainer.train()
    trainer.save_model(args.model_dir)

## 3) Kick-off the fine-tuning job
First we create a [SageMaker Estimator](https://sagemaker.readthedocs.io/en/stable/api/training/estimators.html) with all the parameters we need to launch a training job.

It takes ~27 mins to fine-tune a Llama3.2-1B model using 1 trn1.2xlarge. This time includes 1/ code initialization; 2/ dependencies installing; 3/ model fine-tuning; 4/ trained model uploading to S3.

### 3.1) SageMaker estimator

In [ ]:
import json
import logging
from sagemaker.pytorch import PyTorch

tp_degree=1
batch_size=1
max_seq_len=512
max_prompt_length=256

# ATTENTION: To use llama2 you need to pass HF_TOKEN of an account
# with permission to download Llama2 weights, otherwise the training will fail
ARENA_MAX,model_id=128,"meta-llama/Llama-3.2-1B"

# the default cache repo points to a public / read-only cache
# You can point it to your own repo, but make sure you properly defined the HF token in the HF_TOKEN (above)
CUSTOM_CACHE_REPO="aws-neuron/optimum-neuron-cache"

instance_type='ml.trn1.2xlarge'

hyperparameters={
    "epochs": 3,
    "num_samples": 1024,
    "max_seq_len": max_seq_len,
    "max_prompt_len": max_prompt_length,
    "tp_size": tp_degree,
    "pp_size": 1,
    "zero_1": True,
    "bf16": True,
    "eval_batch_size": batch_size,
    "train_batch_size": batch_size,
    "model_id": model_id
}

if HF_TOKEN and len(HF_TOKEN) > 3:
    hyperparameters["hf_token"]= HF_TOKEN

print(f"Instance type: {instance_type}\nHyperparameters: {hyperparameters}")
estimator = PyTorch(
    entry_point="train.py", # Specify your train script
    source_dir="src",
    role=role,
    sagemaker_session=sess,    
    instance_count=1,
    instance_type=instance_type,
    output_path=f"s3://{bucket}/output",
    disable_profiler=True,
    #input_mode='FastFile', # makes FS read-only
    disable_output_compression=True,
    
    image_uri=f"763104351884.dkr.ecr.{region}.amazonaws.com/pytorch-training-neuronx:2.1.2-neuronx-py310-sdk2.20.0-ubuntu20.04",
    
    volume_size = 512,
    distribution={
        "torch_distributed": {
            "enabled": True
        }
    },
    environment={
        # Uncomment the following line to precompile the cache files
        #"RUN_NEURON_PARALLEL_COMPILE": "1",
        "OMP_NUM_THREADS": "1",
        "FI_EFA_FORK_SAFE": "1",
        "FI_EFA_USE_DEVICE_RDMA": "1",
        "FI_PROVIDER": "efa",
        "XLA_DOWNCAST_BF16": "1",
        "NEURON_FUSE_SOFTMAX": "1",
        "NEURON_RT_ASYNC_EXEC_MAX_INFLIGHT_REQUESTS": "5",
        
        "NEURON_RT_STOCHASTIC_ROUNDING_EN": "1",
        "CUSTOM_CACHE_REPO": CUSTOM_CACHE_REPO,
        "MALLOC_ARENA_MAX": str(ARENA_MAX), # required to avoid OOM
        "NEURON_CC_FLAGS": "--retry_failed_compilation --distribution-strategy=llm-training --enable-saturate-infinity"
    },
    hyperparameters=hyperparameters,
    metric_definitions=[
        {"Name": "train_loss", "Regex": "'loss.:\S*(.*?),"},
        {"Name": "it_per_sec", "Regex": ",\S*(.*?)it.s."},
    ]
)

### 3.2) Launch a SageMaker training job
This will take ~25mins

In [ ]:
from sagemaker.inputs import TrainingInput

estimator.fit({"train": train_path})

### 3.3) Save some parameters for the next notebook

In [ ]:
with open("training_job_name.txt", "w") as f:
    f.write(estimator._current_job_name)
    f.write("\n")
    f.write(HF_TOKEN)
    f.write("\n")

**[Now, go to the next notebook: Deploy the fine-tuned model](02_DeployModel.ipynb)**